In [35]:
import pandas as pd
import re

# ---------------- LOAD CSV ----------------
df = pd.read_csv("/Users/anandhu/Downloads/datatable_export.csv")

# ---------------- COLUMN MAP ----------------
COLUMN_MAP = {
    "SKU": "sku",
    "Product Name": "product_name",
    "Category": "category",
    "Location(s)": "locations",
    "Total Stock": "total_stock",
    "Discontinued": "discontinued",
    "hidden barcode unit": "hidden_barcode_unit",
    "hidden barcode outer": "hidden_barcode_outer"
}

filtered_df = df[list(COLUMN_MAP.keys())].rename(columns=COLUMN_MAP)

# ---------------- DISCONTINUED FILTER ----------------
filtered_df["discontinued"] = (
    filtered_df["discontinued"]
    .astype(str)
    .str.strip()
    .str.lower()
)

filtered_df = filtered_df[filtered_df["discontinued"] == "n"]

# ---------------- ELECTRA (NO REPLEN) ----------------
def extract_valid_electra_locations(location):
    if not isinstance(location, str):
        return None
    location = location.upper()
    matches = re.findall(
        r"(ELECTRA\s+(?!REPLEN)[A-Z0-9\-]+)",
        location
    )
    return ", ".join(matches) if matches else None

filtered_df["locations"] = filtered_df["locations"].apply(extract_valid_electra_locations)
filtered_df = filtered_df[filtered_df["locations"].notna()]

# ---------------- KEEP ONLY P Q R S T AISLES ----------------
def keep_only_pqrst_aisles(location):
    if not isinstance(location, str):
        return None
    location = location.upper()
    matches = re.findall(
        r"(ELECTRA\s+[PQRST]\d+-[A-Z]\d+)",
        location
    )
    return ", ".join(matches) if matches else None

filtered_df["locations"] = filtered_df["locations"].apply(keep_only_pqrst_aisles)
filtered_df = filtered_df[filtered_df["locations"].notna()]

# ---------------- CATEGORY CLEAN ----------------
filtered_df["category"] = filtered_df["category"].astype(str).str.strip()

# ---------------- SPLIT LOCATIONS ----------------
filtered_df["location"] = filtered_df["locations"].str.split(",")
expanded_df = filtered_df.explode("location")

expanded_df["location"] = expanded_df["location"].str.strip()
expanded_df = expanded_df.drop(columns=["locations"])

# ---------------- PARSE AISLE / RACK / SLOT ----------------
def parse_location(location):
    if not isinstance(location, str):
        return pd.Series([None, None, None])

    location = location.replace("ELECTRA", "").strip()

    if "-" not in location:
        return pd.Series([None, None, None])

    aisle, rack_slot = location.split("-", 1)
    aisle = aisle.strip()

    match = re.match(r"([A-Z])(\d+)", rack_slot.strip())
    if not match:
        return pd.Series([aisle, None, None])

    rack = match.group(1)
    slot = match.group(2)

    return pd.Series([aisle, rack, slot])

expanded_df[["aisle", "rack", "slot"]] = expanded_df["location"].apply(parse_location)
# Enforce aisle, rack, slot for ALL categories
expanded_df = expanded_df[
    expanded_df["aisle"].notna() &
    expanded_df["rack"].notna() &
    expanded_df["slot"].notna()
]

# ---------------- SORT ALPHABETICALLY ----------------
expanded_df = expanded_df.sort_values(
    by=["aisle", "rack", "slot"]
).reset_index(drop=True)

# ---------------- SAVE BY CATEGORY ----------------
CATEGORY_FILES = {
    "Pre-filled Pod Kits": "prefilled_pod_kits_locations.csv",
    "Pre-filled Pods": "prefilled_pods_locations.csv",
    "E-liquids - Nic Salts": "eliquids_nic_salts_locations.csv",
    "Nicotine Pouches": "nicotine_pouches_locations.csv"
}

for category, filename in CATEGORY_FILES.items():
    category_df = expanded_df[expanded_df["category"] == category]
    if not category_df.empty:
        category_df.to_csv(f"/Users/anandhu/Downloads/{filename}", index=False)
        print(f"✅ Saved {len(category_df)} rows → {filename}")
    else:
        print(f"⚠️ No data for category: {category}")

# ---------------- SAVE MASTER CLEAN FILE ----------------
expanded_df.to_csv(
    "/Users/anandhu/Downloads/All_products_clean_with_locations.csv",
    index=False
)

print("✅ Final master CSV saved")
print("Total rows:", len(expanded_df))

✅ Saved 1053 rows → prefilled_pod_kits_locations.csv
✅ Saved 675 rows → prefilled_pods_locations.csv
✅ Saved 433 rows → eliquids_nic_salts_locations.csv
✅ Saved 192 rows → nicotine_pouches_locations.csv
✅ Final master CSV saved
Total rows: 2437
